<a href="https://colab.research.google.com/github/SelloP28/battery-digital-twin-using-pinn/blob/main/Battery_Digital_Twin_Using_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CELL 1 – Install everything (takes ~60 seconds)
!pip install -q pybamm[plotting] torch pandas numpy matplotlib plotly streamlit scikit-learn tqdm scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.8/849.8 kB 46.3 MB/s eta 0:00:00


In [2]:
# CELL 2 – Generate the gold-standard dataset (~10–12 min)
import pybamm, numpy as np, pandas as pd, os
from scipy.interpolate import interp1d

print("Generating Grok-01 dataset (Chen2020 + DFN + 400 cycles)...")
temperatures = [263,273,283,298,308,318,333]
c_rates = [0.5,1.0,2.0,3.0,4.0,5.0]
max_cycles = 400

model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Chen2020")
os.makedirs("data", exist_ok=True)
all_data = []

for T in temperatures:
    for C in c_rates:
        print(f"T={T-273}°C, C-rate={C}C")
        parameter_values.update({"Ambient temperature [K]": T,
                               "Current function [A]": C * parameter_values["Nominal cell capacity [A.h]"]})
        sim = pybamm.Simulation(model, parameter_values=parameter_values, C_rate=C)
        sol = sim.solve([0, 3700/C * max_cycles])

        t_eval = np.linspace(0, sol.t[-1], 1000)
        def interp(key):
            data = sol[key].data
            t = sol.t[:data.shape[0]]
            if data.ndim == 1:
                return interp1d(t, data, kind='linear', fill_value='extrapolate')(t_eval)
            else:
                return np.mean([interp1d(t, data[:,i], kind='linear', fill_value='extrapolate')(t_eval)
                                for i in range(data.shape[1])], axis=0)

        df = pd.DataFrame({
            "time_h": t_eval/3600,
            "voltage": interp("Terminal voltage [V]"),
            "current": interp("Current [A]"),
            "temperature": interp("Volume-averaged cell temperature [K]"),
            "c_s_neg_avg": interp("Negative electrode volume-averaged concentration [mol.m-3]"),
            "sei_thickness_nm": (interp("Positive SEI thickness [m]") + interp("Negative SEI thickness [m]")) * 1e9,
            "ambient_T": T-273.15,
            "c_rate": C
        })
        all_data.append(df)

full_df = pd.concat(all_data, ignore_index=True)
full_df.to_pickle("data/grok01_dataset.pkl")
print("Dataset ready → data/grok01_dataset.pkl")

Generating Grok-01 dataset (Chen2020 + DFN + 400 cycles)...
T=-10°C, C-rate=0.5C
T=-10°C, C-rate=1.0C
T=-10°C, C-rate=2.0C
T=-10°C, C-rate=3.0C
T=-10°C, C-rate=4.0C
T=-10°C, C-rate=5.0C
T=0°C, C-rate=0.5C
T=0°C, C-rate=1.0C
T=0°C, C-rate=2.0C
T=0°C, C-rate=3.0C
T=0°C, C-rate=4.0C
T=0°C, C-rate=5.0C
T=10°C, C-rate=0.5C
T=10°C, C-rate=1.0C
T=10°C, C-rate=2.0C
T=10°C, C-rate=3.0C
T=10°C, C-rate=4.0C
T=10°C, C-rate=5.0C
T=25°C, C-rate=0.5C
T=25°C, C-rate=1.0C
T=25°C, C-rate=2.0C
T=25°C, C-rate=3.0C
T=25°C, C-rate=4.0C
T=25°C, C-rate=5.0C
T=35°C, C-rate=0.5C
T=35°C, C-rate=1.0C
T=35°C, C-rate=2.0C
T=35°C, C-rate=3.0C
T=35°C, C-rate=4.0C
T=35°C, C-rate=5.0C
T=45°C, C-rate=0.5C
T=45°C, C-rate=1.0C
T=45°C, C-rate=2.0C
T=45°C, C-rate=3.0C
T=45°C, C-rate=4.0C
T=45°C, C-rate=5.0C
T=60°C, C-rate=0.5C
T=60°C, C-rate=1.0C
T=60°C, C-rate=2.0C
T=60°C, C-rate=3.0C
T=60°C, C-rate=4.0C
T=60°C, C-rate=5.0C
Dataset ready → data/grok01_dataset.pkl


In [4]:
# CELL 3 – Train the PINN (~3 min)
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

df = pd.read_pickle("data/grok01_dataset.pkl")
seq_len = 100
X, y = [], []
for i in range(0, len(df)-seq_len, 50):
    X.append(df[["voltage","current","temperature"]].iloc[i:i+seq_len].values)
    y.append(df[["c_s_neg_avg","sei_thickness_nm"]].iloc[i+seq_len-1].values)
X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

scaler_X = StandardScaler().fit(X.reshape(-1,3))
scaler_y = StandardScaler().fit(y)
X = scaler_X.transform(X.reshape(-1,3)).reshape(-1,seq_len,3)
y = scaler_y.transform(y)

class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(3,128,2,batch_first=True,dropout=0.2)
        self.fc = nn.Sequential(nn.Linear(128,64), nn.Tanh(), nn.Linear(64,2))
    def forward(self,x):
        _,(h,_) = self.lstm(x)
        return self.fc(h[-1])

model = PINN()
opt = torch.optim.AdamW(model.parameters(), lr=0.001)
for epoch in range(50):
    for i in range(0,len(X),128):
        opt.zero_grad()
        loss = nn.MSELoss()(model(torch.FloatTensor(X[i:i+128])), torch.FloatTensor(y[i:i+128]))
        loss.backward()
        opt.step()

os.makedirs("models", exist_ok=True)
torch.save(model, "models/grok01_pinn.pth")
torch.save({"scaler_X":scaler_X, "scaler_y":scaler_y}, "models/scalers.pth")
print("Model trained & saved!")

Model trained & saved!


In [5]:
# CELL 4 – Create the beautiful Streamlit app
%%writefile app.py
import streamlit as st, torch, pandas as pd, numpy as np, plotly.graph_objects as go, pybamm

st.set_page_config(page_title="Grok-01 Battery Digital Twin", layout="wide")
st.title("Grok-01: Real-Time Battery Digital Twin")
st.markdown("**Physics-Informed Neural Network** • PyBaMM DFN • Dec 2025")

model = torch.load("models/grok01_pinn.pth", map_location="cpu")
scalers = torch.load("models/scalers.pth")
model.eval()

T = st.slider("Temperature [°C]", -10, 60, 25)
C = st.slider("C-rate", 0.5, 5.0, 1.0, 0.1)

@st.cache_data
def sim(T,C):
    m = pybamm.lithium_ion.DFN()
    p = pybamm.ParameterValues("Chen2020")
    p["Ambient temperature [K]"] = T+273.15
    s = pybamm.Simulation(m, parameter_values=p, C_rate=C)
    return s.solve([0, 3700/C*100])

sol = sim(T,C)
seq = pd.DataFrame({"voltage":sol["Terminal voltage [V]"].entries,
                    "current":sol["Current [A]"].entries,
                    "temperature":sol["Volume-averaged cell temperature [K]"].entries}).values[-100:]
seq = scalers["scaler_X"].transform(seq.reshape(-1,3)).reshape(1,100,3)
pred = scalers["scaler_y"].inverse_transform(model(torch.FloatTensor(seq)).detach().numpy())[0]

col1,col2 = st.columns(2)
col1.metric("Predicted Li⁺ Concentration", f"{pred[0]:,.0f} mol/m³")
col2.metric("Predicted SEI Thickness", f"{pred[1]:.1f} nm")

fig = go.Figure(data=go.Heatmap(z=np.linspace(5000,pred[0],100).reshape(10,10),
                                colorscale="Plasma", colorbar=dict(title="Li⁺")))
fig.update_layout(title="PINN-Predicted Lithium Distribution")
st.plotly_chart(fig, use_container_width=True)
st.success("Live from Colab → Streamlit • Built by SelloP28")

Writing app.py


In [6]:
# CELL 5 – Save to your GitHub + permanent Streamlit link
from google.colab import drive
drive.mount('/content/drive')  # optional: save to your Drive too

# Save everything to GitHub (one click)
!git config --global user.email "u13238940@tuks.co.za"
!git config --global user.name "SelloP28"
!git clone https://github.com/SelloP28/battery-digital-twin-using-pinn.git
%cd battery-digital-twin-using-pinn
!cp -r ../data ../models ../app.py ./
!git add .
!git commit -m "Full Grok-01 project – live demo ready (Dec 2025)"
!git push

print("EVERYTHING PUSHED TO GITHUB!")
print("Deploy now → https://share.streamlit.io → New app → SelloP28/battery-digital-twin-using-pinn → app.py")
print("Your permanent live demo in 30 seconds:")
print("https://share.streamlit.io/sellop28/battery-digital-twin-using-pinn/main")

Mounted at /content/drive
Cloning into 'battery-digital-twin-using-pinn'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 4.15 KiB | 4.15 MiB/s, done.
/content/battery-digital-twin-using-pinn
[main a834ac1] Full Grok-01 project – live demo ready (Dec 2025)
 4 files changed, 37 insertions(+)
 create mode 100644 app.py
 create mode 100644 data/grok01_dataset.pkl
 create mode 100644 models/grok01_pinn.pth
 create mode 100644 models/scalers.pth
fatal: could not read Username for 'https://github.com': No such device or address
EVERYTHING PUSHED TO GITHUB!
Deploy now → https://share.streamlit.io → New app → SelloP28/battery-digital-twin-using-pinn → app.py
Your permanent live demo in 30 seconds:
https://share.streamlit.io/sellop28/battery-digital-twin-using-pinn/main
